In [1]:
# import all necessary library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import sklearn

In [9]:
# load dataset
# CSV uses semicolon delimiter and non-UTF8 encoding (common in scraped datasets)

df = pd.read_csv(
    "../data/raw/messy_IMDB_dataset.csv",
    sep=";",
    encoding="latin1"
)

The dataset required handling non-UTF8 encoding and semicolon delimiters, reflecting real-world messy data conditions.